In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import time
import random
import copy
import math


In [260]:
# 现在要对5个州做一个总的用水用电的水分配方案（分别给多少水，给多少电 -> 存多少水）
# a = 5个州的分别的总用水需求，b = 5个州分别的用电需求 -> 转化为5个州的存水需求 w
# 求最优的a、w，再计算得到最优的a、b
# 最优的评判：满足总和小于等于 M_v + P_v 的前提下，得分score()最高
# A = 5州分别实际总用水， B = 5州分别实际总用电
# (2015 data)
M_v = 4207.6251
P_v = 3212.9429
N = int(M_v + P_v)
bitlength = N.bit_length()
A = eval("""[4632.39
20171.5
7849.86
2203.23
9412.49]""".replace('\n', ','))
B = eval("""316.5298945
480.7151231
147.2743259
94.57796014
151.905041""".replace('\n', ','))
A, B = np.array(A), np.array(B)
A = A / 1e3
B = B / 365

N_GENERATIONS=100

In [261]:
# 因为目前只有两个水坝分别的水电转换的关系式，应该需要再拟合一个总的：y = 0.0010x + 26.2830
def w2e(x): return 0.0010 * x + 26.2830
def e2w(y): return (y - 26.2830) / 0.0010

a_prime = np.append(A, A+1)
lamb = 1e-7
m=1
def score(ai, a_primei, m):
    value = m / a_primei - m / ai
    if value == 0:
        value += lamb
    else:
        value -= lamb
    return value


def evaluate(a, a_prime, m):
    product = 1
    for i in range(len(a)):
        product = product + score(a[i], a_prime[i], m)
    return math.pow(product, 0.5)


In [262]:
def init_random():
    x, tmp = [], []
    for i in range(9):
        r = random.randint(0, N)
        if r not in tmp and r != 0 and r != N:
            tmp.append(r)
    tmp.sort()
    x.append(tmp[0])
    for i in range(1, 9):
        x.append(tmp[i]-tmp[i-1])
    x.append(N-tmp[-1])
    return x

def swap(x):
    new_x = [0] * len(x)
    x_copy = copy.copy(x)
    np.random.shuffle(x_copy)
    bitlength = N.bit_length()
    for i in range(0, len(x_copy), 2):
        s = random.randint(0, bitlength-1)
        new_x[x.index(x_copy[i])] = int((bin(x_copy[i])[2:]).zfill(bitlength)[:s] + (bin(x_copy[i+1])[2:]).zfill(bitlength)[s:],2)
        new_x[x.index(x_copy[i+1])] = int(bin(x_copy[i+1])[2:].zfill(bitlength)[:s] + bin(x_copy[i])[2:].zfill(bitlength)[s:],2)
    return new_x

def mutation(x):
    new_x = [0] * len(x)
    x_copy = copy.copy(x)
    for i in range(len(x_copy)):
        r = random.randint(0, bitlength-1)
        x_copy_bin = (bin(x_copy[i])[2:]).zfill(bitlength)
        x_copy_bin = x_copy_bin[:r] + str((int(x_copy_bin[r])+1) % 2) + x_copy_bin[r+1:]
        new_x[i] = int(x_copy_bin, 2)
    return new_x

def check(x):
    if 0 in x:
        return False
    s = sum(x)
    if s > N:
        return False
    else:
        return True

In [263]:
best_score = 0
next_x = init_random()
for _ in range(N_GENERATIONS):
    x = next_x
    x_score = evaluate(x, a_prime, m)
    if x_score > best_score:
        best_score = x_score
        best = x
    y = swap(x)
    if not check(y):
        y = x
    y_score = evaluate(y, a_prime, m)
    if y_score > best_score:
        best_score = y_score
        best = y
    z = mutation(y)
    if not check(z):
        z = y
    z_score = evaluate(z, a_prime, m)
    if x_score > best_score:
        best_score = z_score
        best = z
    next_x = z
print(best)

[470, 437, 449, 569, 646, 144, 281, 2169, 295, 920]
